## Defining a minimal Medium

### Import and methods

In [1]:
import cobra
import pandas as pd

model_gf = cobra.io.read_sbml_model("2.2/finegoldia_magna_ATCC_29328_2.2.fo.ch.mp.mcb.lt.re.ar.gpr.pw.gf1.gfmm.gf2.circ.xml")

unwanted_metabolites = ["EX_o2_e"]

# import medium
snm3 = pd.read_csv("SNM3.csv", sep="\t")
snm3_dict = {f"EX_{met['BiGG']}_e" : 10.0 for i,met in snm3.iterrows()}
snm3_dict = {k: v for k,v in snm3_dict.items() if k not in unwanted_metabolites}
            
# For all exchanges open
for reac in model_gf.exchanges:
    if reac.id in unwanted_metabolites:    # elimintes unwanted metabolites (O2)
        reac.lower_bound = 0.0
    else:
        reac.lower_bound = -1000.0

# Define SNM3 medium
#for reac in model_gf.exchanges:
#    if reac.id in snm3_dict: 
#        reac.lower_bound = -10.0
#    else:
#        reac.lower_bound = 0.0

In [2]:
def find_conpro(model, metabolite, conpro):  # finds the metabolites that are produced/consumed from this metabolite
    r_query = []
    for r in model.metabolites.get_by_id(metabolite).reactions:
        if conpro == "produced" and r.get_coefficient(metabolite) < 0:
            r_query += [m for m,stoi in r.metabolites.items() if stoi > 0]
        elif conpro == "consumed" and r.get_coefficient(metabolite) > 0:
            r_query += [m for m,stoi in r.metabolites.items() if stoi < 0]
        elif conpro != "produced" or conpro == "consumed":
            print(f"Wrong conpro argument {conpro}, use one of: produced, consumed")
    return list(set(r_query))

def tree_metabolite(model, metabolite, conpro, depth, reac_thresh):  # makes a tree out of produced/consumed metabolites { {}, {} }
    if depth == 0:
        return None
    sparse_metab = {m.id for m in find_conpro(model, metabolite, conpro) if len(m.reactions) <= reac_thresh}
    return {m : tree_metabolite(model, m, conpro, depth-1, reac_thresh) for m in sparse_metab}
                
def tree_str(nested_tree, direction = ">", delimiter = "|--", depth = 0):  # makes a tree to a string
    t_str = ""
    for parent, child in nested_tree.items():
        if not child:
            t_str = t_str + f"{delimiter * depth}{direction}{parent}\n"
        else:
            t_str = t_str + f"{delimiter * depth}{direction}{parent}\n{tree_str(child, direction, delimiter, depth + 1)}"
    return t_str

### 1. Defining a big medium with different growth-rates

In [3]:
growth_rates = [gr / 10 for gr in range(1, 22)]
minmeds_dict = {gr: cobra.medium.minimal_medium(model_gf, gr, minimize_components = True) for gr in growth_rates}
minmeds_df = pd.concat(minmeds_dict, axis=1)

In [4]:
minmeds_df

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,...,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1
EX_14glucan_e,47.777364,NaN,NaN,NaN,NaN,NaN,157.366207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.344682,NaN,NaN,NaN,146.931629
EX_LalaDgluMdapDala_e,0.001000,0.002000,0.003000,0.004000,0.005000,NaN,0.007000,NaN,NaN,NaN,...,NaN,0.013000,NaN,0.015000,0.016000,0.017000,0.018000,NaN,NaN,NaN
EX_arg__L_e,0.051685,0.103369,0.155054,0.206739,0.258424,0.310108,0.361793,0.413478,0.465162,4.168287,...,0.620216,0.671901,0.723586,0.775270,0.826955,0.878640,0.930325,6.341684,3.486533,1.085379
EX_bz_e,0.000010,0.000020,0.000030,0.000040,0.000050,0.000060,0.000070,0.000080,0.000090,0.000806,...,0.000120,0.000130,0.000140,0.000150,0.000160,0.000170,0.000180,0.001227,0.000675,0.000210
EX_ca2_e,0.000521,0.001041,0.001562,0.002082,0.002603,0.003123,0.003644,0.004164,0.004685,0.041977,...,0.006246,0.006766,0.007287,0.007807,0.008328,0.008848,0.009369,0.063865,0.035112,0.010930
EX_cl_e,0.000521,0.001041,0.001562,0.002082,0.002603,0.003123,0.003644,0.004164,0.004685,0.041977,...,0.006246,0.006766,0.007287,0.007807,0.008328,0.008848,0.009369,0.063865,0.035112,0.010930
EX_cobalt2_e,0.000010,0.000020,0.000030,0.000040,0.000050,0.000060,0.000070,0.000080,0.000090,0.000806,...,0.000120,0.000130,0.000140,0.000150,0.000160,0.000170,0.000180,0.001227,0.000675,0.000210
EX_cu2_e,0.000071,0.000142,0.000213,0.000284,0.000354,0.000425,0.000496,0.000567,0.000638,0.005718,...,0.000851,0.000922,0.000993,0.001063,0.001134,0.001205,0.001276,0.008699,0.004783,0.001489
EX_fe2_e,0.000672,NaN,0.002014,0.002686,NaN,NaN,0.004700,0.005729,0.006043,0.054155,...,NaN,0.008729,0.009401,NaN,NaN,0.011415,NaN,NaN,0.045298,NaN
EX_fe3pyovd_kt_e,0.000781,297.891073,NaN,0.003123,0.007261,NaN,NaN,NaN,NaN,NaN,...,NaN,0.010150,0.010931,0.021784,NaN,NaN,0.026141,0.178196,NaN,0.030498


26dap__M_e | LalaDgluMdapDala_e  
( fe3_e |fe3pyovd_kt_e ) & ( peamn_e & | fe2_e )  
tyr__L_e   | 4hphac_e  

EX_peamn_e, EX_fe3pyovd_kt_e intermittedly there, both involved in iron  
EX_o2_e biologically not in FM -> take out  
  
--- found out via tree making ---

In [5]:
metab_id = "fe3pyovd_kt_e"
tm = { metab_id: tree_metabolite(model_gf, metab_id, "produced", 4, 10) }
print(tree_str(tm, ">"))

>fe3pyovd_kt_e
|-->fe3pyovd_kt_p
|--|-->fe2_p
|--|--|-->fe3_p
|--|--|--|-->fe3_c
|--|--|-->fe2_c
|--|-->pqq_p
|--|--|-->nh4_p
|--|--|-->pacald_p
|--|--|-->pqqh2_p
|--|--|--|-->fe2_p
|--|--|--|-->pqq_p
|--|--|--|-->pyovd_kt_p
|--|-->pyovd_kt_p
|--|--|-->pyovd_kt_e
|--|--|--|-->fe3pyovd_kt_e



In [6]:
# => minimal medium: "EX_4hphac_e", "EX_LalaDgluMdapDala_e", "EX_peamn_e", "EX_fe3pyovd_kt_e" entfernt,
# da Einfache Metalle, Aminosäuren und einfachere Strukturen bevorzugt wurden
minimal_medium_all = list(minmeds_df.index)
for x in ["EX_4hphac_e", "EX_LalaDgluMdapDala_e", "EX_peamn_e", "EX_fe3pyovd_kt_e"]:
    minimal_medium_all.remove(x)
minimal_medium_all = {r: 10.0 for r in minimal_medium_all}
print(len(minimal_medium_all))
model_gf.medium = minimal_medium_all
model_gf.slim_optimize()

37


4.213172966615248

In [7]:
# adding SNM3 medium to minimal medium
snm3_dict = {k: v for k,v in snm3_dict.items() if k in model_gf.reactions}
minimal_and_snm3_medium = minimal_medium_all.copy()
minimal_and_snm3_medium.update(snm3_dict)
model_gf.medium = minimal_and_snm3_medium
model_gf.slim_optimize()

4.508748020468335

### 2. Taking one out a a time until no more growth

#### Helping methods

In [8]:
def littlest_growth_diff(model, medium:dict, threshold):
    model_acc = model.copy()
    medium = medium.copy()
    model_acc.medium = medium
    
    prior_growth = model_acc.slim_optimize()
    diff_dict = {}
    for metab, flux in medium.items():
        new_medium = medium.copy()
        new_medium.pop(metab)
        model_acc.medium = new_medium
        diff_dict[prior_growth - model_acc.slim_optimize()] =  metab
    
    metab_smallest_diff = diff_dict[ min( diff_dict.keys() ) ]
    new_medium = medium.copy()
    new_medium.pop(metab_smallest_diff)
    model_acc.medium = new_medium
    if model_acc.slim_optimize() <= threshold:
        print("No more growth")
        return None
    
    return metab_smallest_diff

def eliminate_until(model, medium:dict, threshold):
    model_acc = model.copy()
    elim_metab = littlest_growth_diff(model, medium, threshold)
    if elim_metab is None:
        return medium
    else:
        new_medium = medium.copy()
        new_medium.pop(elim_metab)
        return eliminate_until(model, new_medium, threshold)

#### 2.1 Only for minimal medium

In [9]:
print(f"Before minimization: {len(minimal_medium_all)}")
mini_mini_all = eliminate_until(model_gf, minimal_medium_all, 0.01)
print(f"After minimization: {len(mini_mini_all)}")

Before minimization: 37
No more growth
After minimization: 29


In [10]:
print(mini_mini_all.keys())

dict_keys(['EX_arg__L_e', 'EX_bz_e', 'EX_ca2_e', 'EX_cl_e', 'EX_cobalt2_e', 'EX_cu2_e', 'EX_fe2_e', 'EX_gthrd_e', 'EX_hdca_e', 'EX_his__L_e', 'EX_ile__L_e', 'EX_k_e', 'EX_leu__L_e', 'EX_lys__L_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_nmn_e', 'EX_ocdca_e', 'EX_pnto__R_e', 'EX_pser__L_e', 'EX_ribflv_e', 'EX_so4_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_val__L_e', 'EX_zn2_e', 'EX_fe3_e', 'EX_26dap__M_e', 'EX_malthx_e'])


#### 2.2 For minimal medium + SNM3

In [11]:
print(f"Before minimization: {len(minimal_and_snm3_medium)}")
mini_mini_snm3 = eliminate_until(model_gf, minimal_and_snm3_medium, 0.01)
print(f"After minimization: {len(mini_mini_snm3)}")

Before minimization: 58
No more growth
After minimization: 30


### 3. Grow & report final media

In [12]:
# Normal minimal medium
model_gf.medium = mini_mini_all
model_gf.slim_optimize()

1.112913613822898

In [13]:
# SNM3 minimal medium
model_gf.medium = mini_mini_snm3
model_gf.slim_optimize()

1.1476182009090696

#### 3.1 Difference of media

The final media differ in two ways: 

In [14]:
# Difference between two media:
combined_set = set(mini_mini_snm3.keys()) | set(mini_mini_all.keys())
print(combined_set - set(mini_mini_all.keys()))
print(combined_set - set(mini_mini_snm3.keys()))

{'EX_cys__L_e', 'EX_glu__L_e'}
{'EX_gthrd_e'}


#### 3.2 Missing in SNM3

The final difference is, that Reduced glutathione (gthrd_e) is in the standard minimal medium, while L-Glutamate (glu__L_e), L-Cysteine (cys__L_e) remain in the snm3 combined minimal medium

In [15]:
# Difference between minmed and snm3:
print(set(mini_mini_all.keys() - set(snm3_dict.keys())))
# Difference between snm3 minmed and snm3:
print(set(mini_mini_snm3.keys() - set(snm3_dict.keys())))

{'EX_nmn_e', 'EX_bz_e', 'EX_ocdca_e', 'EX_gthrd_e', 'EX_26dap__M_e', 'EX_pser__L_e', 'EX_tyr__L_e', 'EX_ile__L_e', 'EX_malthx_e', 'EX_hdca_e'}
{'EX_nmn_e', 'EX_bz_e', 'EX_ocdca_e', 'EX_26dap__M_e', 'EX_pser__L_e', 'EX_tyr__L_e', 'EX_ile__L_e', 'EX_malthx_e', 'EX_hdca_e'}


The 10 missing metabolites in the SNM3 medium are: 
- Meso-2,6-Diaminoheptanedioate 26dap__M_e
- Octadecanoate (n-C18:0) ocdca_e
- Reduced glutathione gthrd_e
- L-Tyrosine tyr__L_e
- L-Isoleucine ile__L_e
- Maltohexaose malthx_e
- Benzoate bz_e
- NMN C11H14N2O8P nmn_e
- Hexadecanoate (n-C16:0) hdca_e
- O-Phospho-L-serine pser__L_e  

which can be decreased through the snm3 minimal medium to 9 because Reduced glutathione (gthrd_e) can be avoided

In [48]:
model_gf = cobra.io.read_sbml_model("2.2/finegoldia_magna_ATCC_29328_2.2.fo.ch.mp.mcb.lt.re.ar.gpr.pw.gf1.gfmm.gf2.circ.xml")

# Export as table
exp_df = pd.DataFrame(mini_mini_all.items(),  columns=['reaction', 'flux'])
names = [list(model_gf.reactions.get_by_id(row[1]['reaction']).metabolites.keys())[0].name for row in exp_df.iterrows()]
exp_df["names"] = names
exp_df.to_csv("../Tables/minimal_medium_all.csv")

exp_df = pd.DataFrame(mini_mini_snm3.items(), columns=['reaction', 'flux'])
names = [list(model_gf.reactions.get_by_id(row[1]['reaction']).metabolites.keys())[0].name for row in exp_df.iterrows()]
exp_df["names"] = names
exp_df.to_csv("../Tables/minimal_medium_snm3.csv")

In [45]:
for row in exp_df.iterrows():
    print(row[1]["reaction"])

EX_arg__L_e
EX_bz_e
EX_ca2_e
EX_cl_e
EX_cobalt2_e
EX_cu2_e
EX_fe2_e
EX_gthrd_e
EX_hdca_e
EX_his__L_e
EX_ile__L_e
EX_k_e
EX_leu__L_e
EX_lys__L_e
EX_mg2_e
EX_mn2_e
EX_nmn_e
EX_ocdca_e
EX_pnto__R_e
EX_pser__L_e
EX_ribflv_e
EX_so4_e
EX_trp__L_e
EX_tyr__L_e
EX_val__L_e
EX_zn2_e
EX_fe3_e
EX_26dap__M_e
EX_malthx_e
